In [1]:
def ludecomp_1(a, n):
    a: float
    n: int
    index = [0] * n
    sign: int
    eflag: int
    return a, index, sign, eflag

In [2]:
def lusubst_1(a, n, b, index):
    a: float
    n: int
    b: float
    index: int
    eflag: int
    return b, eflag

In [ ]:
def ludecomp(a, n):
    a: float
    n: int
    sign: int
    eflag: int
    
    #初期値の設定
    index = [0] * n  # 行の入れ替えを記録
    sign = 1         # 置換の符号
    eflag = 0
    
    # スケーリング因子の準備（各行の最大要素を格納）
    vv = [0.0] * n
    for i in range(n):
        big = 0.0
        for j in range(n):
            temp = abs(a[i][j])
            if temp > big: 
                big = temp
        if big == 0.0:
            eflag = 1 # ゼロのみの行がある（特異行列）
            return a, index, sign, eflag
        vv[i] = 1.0 / big

    # Crout法によるLU分解
    for j in range(n):
        for i in range(j):
            sum_val = a[i][j]
            for k in range(i):
                sum_val -= a[i][k] * a[k][j]
            a[i][j] = sum_val
        
        big = 0.0
        for i in range(j, n):
            sum_val = a[i][j]
            for k in range(j):
                sum_val -= a[i][k] * a[k][j]
            a[i][j] = sum_val
            
            # ピボット選択
            dum = vv[i] * abs(sum_val)
            if dum >= big:
                big = dum
                imax = i
        
        # 必要に応じて行を入れ替える
        if j != imax:
            for k in range(n):
                a[imax][k], a[j][k] = a[j][k], a[imax][k]
            sign = -sign
            vv[imax] = vv[j]
        
        index[j] = imax
        
        if a[j][j] == 0.0:
            eflag = 1
            return a, index, sign, eflag
        
        if j != n - 1:
            dum = 1.0 / a[j][j]
            for i in range(j + 1, n):
                a[i][j] *= dum
                
    return a, index, sign, eflag

In [ ]:
def lusubst(a, n, b, index):
    a: float
    n: int
    b: float
    index: int
    eflag: int
    
    #初期値の設定
    eflag = 0
    
    # 前進代入 (Ly = b)
    ii = -1
    for i in range(n):
        ip = index[i]
        sum_val = b[ip]
        b[ip] = b[i]
        if ii >= 0:
            for j in range(ii, i):
                sum_val -= a[i][j] * b[j]
        elif sum_val != 0.0:
            ii = i
        b[i] = sum_val
        
    # 後退代入 (Ux = y)
    for i in range(n - 1, -1, -1):
        sum_val = b[i]
        for j in range(i + 1, n):
            sum_val -= a[i][j] * b[j]
        b[i] = sum_val / a[i][i]
        
    return b, eflag

In [ ]:
import sys

# 行列AとベクトルBの定義
A = [
    [2., 4., 2.],
    [2., 0., 1.],
    [1., 1., 1.]
]
B = [6., 2., 1.]

N: int = 3
sign: int
eflag: int
i: int
j: int
c = [[0] * N for i in range(N)] #すべての要素が0に初期化された正方行列を作成
x = [0] * N
index = [0] * N
det: float

print('A=', A)
print('B=', B)

# LU分解の実行
A, index, sign, eflag = ludecomp(A, N)
if eflag > 0:
    print('Error in ludecomp', eflag)
    print('LU of A=', A)
    sys.exit() #実行を強制終了

# 行列式の計算
det = 1
for i in range(N):
    det = det * A[i][i]
det = det * sign
print('det A=', det)

# 連立方程式 Ax=B の解を求める
x, eflag = lusubst(A, N, B, index)
if eflag > 0:
    print('Error in lusubst', eflag)
    sys.exit() #実行を強制終了
print('x satisfying Ax=B is', x)

# 逆行列 A^(-1) の計算
for j in range(N):
    for i in range(N):
        B[i] = 0
        if i == j:
            B[i] = 1
    # 単位行列の各列を B として代入し、解を求める
    B, eflag = lusubst(A, N, B, index)
    if eflag > 0:
        print('Error in lusubst', eflag)
        sys.exit() #実行を強制終了
    for i in range(N):
        c[i][j] = B[i]

print('A^(-1)=')
print(c)

A= [[2.0, 4.0, 2.0], [2.0, 0.0, 1.0], [1.0, 1.0, 1.0]]
B= [6.0, 2.0, 1.0]
det A= -2.0
x satisfying Ax=B is [3.0, 2.0, -4.0]
A^(-1)=
[[0.5, 1.0, -2.0], [0.5, -0.0, -1.0], [-1.0, -1.0, 4.0]]
